In [14]:
############## NOTICE ################
# 이 셀은 일부분 수정해서 돌려야 합니다.

import os
import yaml
import chatgpt
import perplexity

############ TODO #################
# 1. 기업 맞춤형 메일을 작성해야하므로 타겟 기업에 대한 정보를 글로 긁어와야함. 
# 단순히 투자유치 받았다 이런 기사는 쓸모없고, 기업이 어떤 문제를 어떤 접근방식으로 풀고있는지에 대한 정보가 중요함.

# [TIP!]
# https://www.perplexity.ai 에서 정리해준 정보 사용하기.
# 타겟 기업 홈페이지 링크와 함께 '위 링크를 참고하여 {(1), (2)}' 각각 따로 검색 후 모아서 복사 (특정 링크 기제 시 기업 정보의 질이 매우 올라간다. 대신 제시한 링크 외부의 정보는 이용 못함. 최신 기사 등)
# (1): '기업과 관련된 정보를 비즈니스 모델을 포함하여 조사해줘'
# (2): '기업에서 해결 중일 것으로 추정되는 문제를 최대한 다양하게 근거와 함께 제시해줘'
#  --> sources/SOURCE_target_company_information.txt에 출력 결과 붙여넣기

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
PPLX_API_KEY = os.environ["PPLX_API_KEY"]

with open('INPUT.yaml', encoding='UTF8') as f:
    input_info = yaml.full_load(f)
    
    
if input_info['company'] is None:
    raise Exception("INPUT.yaml 파일에서 기업명 작성해주세요.")     


prompt_company_info_sys = "해당 기업과 관련된 정보 찾아줘. 반드시 검색엔진에 정보가 있으니 관련된 모든 정보 찾아내."



if input_info['homepage_link'] is None:
    prompt_business_model_usr = f"""
    
    {input_info['company']} 기업에서 제공하는 서비스 관련 정보를 비즈니스 모델 위주로 정리해줘.
    
    """
    prompt_solving_problem_usr = f"""
    
    {input_info['company']} 기업에서 해결 중일 것으로 추정되는 문제를 최대한 다양하게 근거와 함께 제시해줘.
    
    """
    
    
else:
    prompt_business_model_usr = f"""
    
    {input_info['homepage_link']}
    
    위 링크를 참조하여 {input_info['company']} 기업에서 제공하는 서비스 관련 정보를 비즈니스 모델 위주로 정리해줘.
    만약 해당 링크가 운영 중이 아니라면 검색엔진에 검색하여 기업 정보를 찾아내.
    
    """
    prompt_solving_problem_usr = f"""
    
    {input_info['homepage_link']}
    
    위 링크를 참조하여 {input_info['company']} 기업에서 해결 중일 것으로 추정되는 문제를 최대한 다양하게 근거와 함께 제시해줘.
    만약 해당 링크가 운영 중이 아니라면 검색엔진에 검색하여 기업 정보를 찾아내.
    
    """
    
# 1) 비즈니스 모델 조사
prompt = [
    {"role":"system", "content":prompt_company_info_sys},
    {"role":"user", "content":prompt_business_model_usr}
]

business_model_info = perplexity.run_chat_completion(PPLX_API_KEY, prompt).choices[0].message.content

# 2) 풀고 있는 문제 조사
prompt = [
    {"role":"system", "content":prompt_company_info_sys},
    {"role":"user", "content":prompt_solving_problem_usr}
]

solving_problem_info = perplexity.run_chat_completion(PPLX_API_KEY, prompt).choices[0].message.content




f= open("sources/SOURCE_target_company_information.txt","w+", encoding='UTF8')

target_company_information = business_model_info + "\n\n" + solving_problem_info
f.write(target_company_information)
f.close()

# 2. 기업명과 도메인은 직접 입력해주세요. gpt한테 추론하도록 시켰는데 기업명은 잘 잡아내지만 도메인 분류는 잘 못해서 수동입력해주는 게 좋을 듯
# --> 아래 TODO 에도 적어놨는데, prompt_usr 중 '기업명'과 '도메인'을 적절하게 수정하면 됩니다.
# 완성된 제안서는 OUTPUT_completed_proposal_letter.txt에서 확인 가능합니다.



################# TODO ######################
# 3. 링크드인의 경우, 이곳에 담당자 정보를 기입해주세요. 영문명 그대로 기입해도 표준 로마자 변환 규칙에 따라 한국어명으로 변환됩니다.
# 콜드메일의 경우, 기입하지않고 그대로 놔둡니다.

personal_info_sys = open("prompts/PROMPT_personal_info_transform.txt", 'r', encoding='UTF8').read()
personal_info_usr = f"""

[담당자 정보]
성함: {input_info['name']}
직함: {input_info['position']}

"""


prompt = [{"role":"system", "content":personal_info_sys},
            {"role":"user", "content":personal_info_usr}
        ]
transformed_personal_info = chatgpt.run_chat_completion(OPENAI_API_KEY, prompt).choices[0].message.content


####################################################################################

summarized_projects = open("sources/SOURCE_summarized_project.txt", 'r', encoding='UTF8').read()
#target_company_information = open("sources/SOURCE_target_company_information.txt", 'r', encoding='UTF8').read()

select_relative_proj_sys = open("prompts/PROMPT_select_relative_project.txt", 'r', encoding='UTF8').read()
select_relative_proj_usr = """
다음 내용을 바탕으로 타겟 기업과 관련된 프로젝트를 뽑아서 출력해줘


[타겟 기업 관련 정보]

{target_company_information}

[지난 여러 기업과 진행한 프로젝트들]

{summarized_projects}

"""

prompt = [
            {"role":"system", "content":select_relative_proj_sys},
            {"role":"user", "content":select_relative_proj_usr.format(target_company_information = target_company_information, summarized_projects = summarized_projects)}
        ]

selected_relative_project = chatgpt.run_chat_completion(OPENAI_API_KEY, prompt).choices[0].message.content




#################################################################################


template = open("sources/SOURCE_proposal_template.txt", 'r', encoding='UTF8').read()
template = template.format(selected_relative_project=selected_relative_project)

prompt_sys = open("prompts/PROMPT_complete_proposal_letter.txt", 'r', encoding='UTF8').read()
prompt_usr = """
다음 내용을 바탕으로 완전한 협업 제안 메일을 발송해줘.

[담당자 정보]

{transformed_personal_info}



[타겟 기업 관련 정보]

{target_company_information}



[관련 프로젝트]

{selected_relative_project}



[제안서 템플릿]

{template}



"""


f= open(f"outputs/{input_info['company']}{("_"+input_info['name']) if input_info['name'] is not None else ""}_completed_proposal_letter.txt","w+", encoding='UTF8')
    
prompt = [{"role":"system", "content":prompt_sys},
            {"role":"user", "content":prompt_usr.format(target_company_information = target_company_information, template = template, transformed_personal_info=transformed_personal_info, selected_relative_project=selected_relative_project)}
        ]
answer = chatgpt.run_chat_completion(OPENAI_API_KEY, prompt)
f.write(answer.choices[0].message.content)

f.close()

'2ha'